In [63]:
import rasterio
import numpy as np
import geopandas as gpd

def veg2fuel(filepath_mat, filepath_arb, dst, box):

    ds_mat = rasterio.open(filepath_mat)
    meta = ds_mat.meta
    transform = meta.get('transform')
    window = rasterio.windows.from_bounds(*b, transform)
    array_mat = ds_mat.read(1, window=window, boundless=True, fill_value=-9999)
    
    ds_arb = rasterio.open(filepath_arb)
    window = rasterio.windows.from_bounds(*b, ds_arb.meta.get('transform'))
    array_arb = ds_arb.read(1, window=window, boundless=True, fill_value=-9999)

    print(array_mat.shape, array_arb.shape, transform)
    print(ds_arb.meta.get('transform'))
    print(window)
    shape = array_mat.shape
    
    new_array = np.zeros([(int(shape[0] / 10)), int(shape[1] / 10)])

    originX = b[0]
    originY = b[3]
    
    for new_col in range(new_array.shape[0]):
        for new_row in range(new_array.shape[1]):
            arb = np.full(100, np.nan)
            mat = np.full(100, np.nan)
            amb = np.full(100, np.nan)
            coef_arb = 0
            coef_amb = 0
            coef_mat = 0
            coef_edif = 0
            for col in range(10):
                for row in range(10):
                    data_arb = array_arb[new_col * 10 + col][new_row * 10 + row]
                    data_mat = array_mat[new_col * 10 + col][new_row * 10 + row]
                    if data_arb != -9999 and data_mat != -9999:
                        amb[col * 10 + row] = data_arb
                        coef_amb += 1
                    if data_mat != -9999 and data_arb == -9999:
                        mat[col * 10 + row] = data_mat
                        coef_mat += 1
                    if data_arb != -9999 and data_mat == -9999:
                        arb[col * 10 + row] = data_arb
                        coef_arb += 1
    
            if coef_arb + coef_mat + coef_amb > 66:
                if coef_arb >= coef_amb and coef_arb >= coef_mat:
                    #                 pto = getCoordinates(new_col, new_row)
                    #                 sp1 = consulta(pto)
                    #                 if sp1 == 21:
                    #                     new_array[new_col, new_row] = 8
                    #                 else:
                    #                     new_array[new_col, new_row] = 9
                    new_array[new_col, new_row] = 9
                elif coef_mat >= coef_amb and coef_mat >= coef_arb:
                    if np.nanmean(mat) < 0.8:
                        new_array[new_col, new_row] = 5
                    elif np.nanmean(mat) < 1.6:
                        new_array[new_col, new_row] = 6
                    else:
                        new_array[new_col, new_row] = 4
                else:
                    new_array[new_col, new_row] = 7
            else:
                #             pto = getCoordinates(new_col, new_row)
                #             clas = consulta2(pto)
                #                 if clas == 300:
                #                     new_array[new_col, new_row] = 0
                #                 else:
                #                     if coef_mat + coef_arb + coef_amb < 33:
                #                         new_array[new_col, new_row] = 1
                #                     else:
                #                         new_array[new_col, new_row] = 2
                if coef_mat + coef_arb + coef_amb < 33:
                    new_array[new_col, new_row] = 1
                else:
                    new_array[new_col, new_row] = 2
    print(new_array.shape)
    meta.update({
        'width': new_array.shape[1],
        'height': new_array.shape[0],
        'transform': transform * transform.scale(10)
    })
    print(meta)
    with rasterio.open(dst, "w", **meta) as dest:
        dest.write(new_array, 1)

In [64]:
z = 1
vuelo = 'vuelo_2'

zonas = gpd.read_file('shp/zonas_estudio.shp')
zona = zonas[zonas.Id == z]
b = zona.iloc[0].geometry.bounds
veg2fuel(f'FINALES/{vuelo}_zona{z}_matorral.tif', f'FINALES/{vuelo}_zona{z}_arbolado.tif', f'fm_{''.join(vuelo.split('_'))}_zona{z}.tif', b)

(270, 872) (270, 872) | 1.00, 0.00, 647408.21|
| 0.00,-1.00, 4721434.85|
| 0.00, 0.00, 1.00|
| 1.00, 0.00, 647408.57|
| 0.00,-1.00, 4721433.17|
| 0.00, 0.00, 1.00|
Window(col_off=-0.3805211342405528, row_off=-0.7382897194474936, width=872.1578509863466, height=269.68799221888185)
(27, 87)
{'driver': 'GTiff', 'dtype': 'float64', 'nodata': -9999.0, 'width': 87, 'height': 27, 'count': 1, 'crs': CRS.from_epsg(25830), 'transform': Affine(10.0, 0.0, 647408.21,
       0.0, -10.0, 4721434.850000001)}
